As seguintes bibliotecas adicionais são necessárias para executar este
notebook. Observe que a execução no Colab é experimental, por favor, relate um Github
questão se você tiver algum problema.


In [ ]:
!pip install d2l==1.0.3


# Representações de codificadores bidirecionais de transformadores (BERT)
:rótulo:`sec_bert`

Introduzimos vários modelos de incorporação de palavras para compreensão da linguagem natural.
Após o pré-treinamento, a saída pode ser considerada uma matriz
onde cada linha é um vetor que representa uma palavra de um vocabulário predefinido.
Na verdade, todos esses modelos de incorporação de palavras são *independentes do contexto*.
Vamos começar ilustrando essa propriedade.


## De independente de contexto para sensível ao contexto

Lembre-se dos experimentos em :numref:`sec_word2vec_pretraining` e :numref:`sec_synonyms`.
Por exemplo, word2vec e GloVe atribuem o mesmo vetor pré-treinado à mesma palavra, independentemente do contexto da palavra (se houver).
Formalmente, uma representação independente de contexto de qualquer token $x$
é uma função $f(x)$ que recebe apenas $x$ como entrada.
Dada a abundância de polissemia e semântica complexa nas línguas naturais,
representações independentes de contexto têm limitações óbvias.
Por exemplo, a palavra "guindaste" em contextos
"um guindaste está voando" e "um motorista de guindaste veio" têm significados completamente diferentes;
assim, a mesma palavra pode receber representações diferentes dependendo do contexto.

Isso motiva o desenvolvimento de representações de palavras *sensíveis ao contexto*,
onde as representações de palavras dependem de seus contextos.
Portanto, uma representação sensível ao contexto do token $x$ é uma função $f(x, c(x))$
dependendo tanto de $x$ quanto de seu contexto $c(x)$.
Representações populares sensíveis ao contexto
incluir TagLM (marcador de sequência aumentada por modelo de linguagem) :cite:`Peters.Ammar.Bhagavatula.ea.2017`,
CoVe (Vetores de Contexto) :cite:`McCann.Bradbury.Xiong.ea.2017`,
e ELMo (Embeddings de Modelos de Linguagem):cite:`Peters.Neumann.Iyyer.ea.2018`.

Por exemplo, tomando a sequência inteira como entrada,
ELMo é uma função que atribui uma representação a cada palavra da sequência de entrada.
Especificamente, o ELMo combina todas as representações da camada intermediária do LSTM bidirecional pré-treinado como a representação de saída.
Em seguida, a representação ELMo será adicionada ao modelo supervisionado existente de uma tarefa posterior
como recursos adicionais, como concatenar a representação ELMo e a representação original (por exemplo, GloVe) de tokens no modelo existente.
Por um lado,
todos os pesos no modelo LSTM bidirecional pré-treinado são congelados após as representações ELMo serem adicionadas.
Por outro lado,
o modelo supervisionado existente é especificamente personalizado para uma determinada tarefa.
Aproveitando diferentes modelos para diferentes tarefas naquele momento,
a adição do ELMo melhorou o estado da arte em seis tarefas de processamento de linguagem natural:
análise de sentimentos, inferência de linguagem natural,
rotulagem de papéis semânticos, resolução de correferências,
reconhecimento de entidade nomeada e resposta a perguntas.


## De específico para tarefa para agnóstico para tarefa

Embora o ELMo tenha melhorado significativamente as soluções para um conjunto diversificado de tarefas de processamento de linguagem natural,
cada solução ainda depende de uma arquitetura *específica da tarefa*.
No entanto, é praticamente não trivial criar uma arquitetura específica para cada tarefa de processamento de linguagem natural.
O modelo GPT (Generative Pre-Training) representa um esforço na concepção
um modelo geral *independente de tarefas* para representações sensíveis ao contexto :cite:`Radford.Narasimhan.Salimans.ea.2018`.
Construído em um decodificador Transformer,
O GPT pré-treina um modelo de linguagem que será usado para representar sequências de texto.
Ao aplicar GPT a uma tarefa downstream,
a saída do modelo de linguagem será alimentada em uma camada de saída linear adicionada
para prever o rótulo da tarefa.
Em nítido contraste com o ELMo que congela os parâmetros do modelo pré-treinado,
O GPT ajusta *todos* os parâmetros no decodificador Transformer pré-treinado
durante o aprendizado supervisionado da tarefa subsequente.
O GPT foi avaliado em doze tarefas de inferência de linguagem natural,
resposta a perguntas, semelhança de frases e classificação,
e melhorou o estado da arte em nove deles com mudanças mínimas
para a arquitetura do modelo.

No entanto, devido à natureza autorregressiva dos modelos de linguagem,
O GPT olha apenas para a frente (da esquerda para a direita).
Nos contextos "fui ao banco para depositar dinheiro" e "fui ao banco para me sentar",
como "banco" é sensível ao contexto à sua esquerda,
O GPT retornará a mesma representação para "banco",
embora tenha significados diferentes.


## BERT: Combinando o melhor dos dois mundos

Como vimos,
O ELMo codifica o contexto bidirecionalmente, mas usa arquiteturas específicas para tarefas;
enquanto o GPT é independente de tarefa, mas codifica o contexto da esquerda para a direita.
Combinando o melhor dos dois mundos,
BERT (Representações de codificadores bidirecionais de transformadores)
codifica o contexto bidirecionalmente e requer mudanças mínimas na arquitetura
para uma ampla gama de tarefas de processamento de linguagem natural :cite:`Devlin.Chang.Lee.ea.2018`.
Usando um codificador Transformer pré-treinado,
BERT é capaz de representar qualquer token com base em seu contexto bidirecional.
Durante a aprendizagem supervisionada de tarefas posteriores,
O BERT é semelhante ao GPT em dois aspectos.
Primeiro, as representações BERT serão alimentadas em uma camada de saída adicional,
com alterações mínimas na arquitetura do modelo dependendo da natureza das tarefas,
como prever para cada token vs. prever para a sequência inteira.
Segundo,
todos os parâmetros do codificador Transformer pré-treinado são ajustados com precisão,
enquanto a camada de saída adicional será treinada do zero.
:numref:`fig_elmo-gpt-bert` descreve as diferenças entre ELMo, GPT e BERT.

![Uma comparação de ELMo, GPT e BERT.](https://github.com/d2l-ai/d2l-pytorch-colab/blob/master/img/elmo-gpt-bert.svg?raw=1)
:label:`fig_elmo-gpt-bert`


O BERT melhorou ainda mais o estado da arte em onze tarefas de processamento de linguagem natural
sob categorias amplas de (i) classificação de texto único (por exemplo, análise de sentimentos), (ii) classificação de pares de textos (por exemplo, inferência de linguagem natural),
(iii) resposta a perguntas, (iv) marcação de texto (por exemplo, reconhecimento de entidade nomeada).
Tudo proposto em 2018,
de ELMo sensível ao contexto para GPT e BERT independentes de tarefas,
O pré-treinamento conceitualmente simples, mas empiricamente poderoso, de representações profundas para linguagens naturais revolucionou soluções para diversas tarefas de processamento de linguagem natural.

No restante deste capítulo,
vamos nos aprofundar no pré-treinamento do BERT.
Quando as aplicações de processamento de linguagem natural são explicadas em :numref:`chap_nlp_app`,
ilustraremos o ajuste fino do BERT para aplicações posteriores.


In [13]:
import torch
from torch import nn
from d2l import torch as d2l

## [**Representação de entrada**]
:rótulo:`subsec_bert_input_rep`

No processamento de linguagem natural,
algumas tarefas (por exemplo, análise de sentimentos) usam um único texto como entrada,
enquanto em algumas outras tarefas (por exemplo, inferência de linguagem natural),
a entrada é um par de sequências de texto.
A sequência de entrada BERT representa de forma inequívoca tanto texto único quanto pares de texto.
No primeiro,
a sequência de entrada BERT é a concatenação de
o token de classificação especial “&lt;cls&gt;”,
tokens de uma sequência de texto,
e o token de separação especial “&lt;sep&gt;”.
Neste último,
a sequência de entrada BERT é a concatenação de
“&lt;cls&gt;”, tokens da primeira sequência de texto,
“&lt;sep&gt;”, tokens da segunda sequência de texto, e “&lt;sep&gt;”.
Distinguiremos consistentemente a terminologia "sequência de entrada BERT"
de outros tipos de "sequências".
Por exemplo, uma *sequência de entrada BERT* pode incluir uma *sequência de texto* ou duas *sequências de texto*.

Para distinguir pares de texto,
os embeddings de segmento aprendidos $\mathbf{e}_A$ e $\mathbf{e}_B$
são adicionados aos embeddings de token da primeira sequência e da segunda sequência, respectivamente.
Para entradas de texto simples, somente $\mathbf{e}_A$ é usado.

O seguinte `get_tokens_and_segments` aceita uma ou duas frases
como entrada, então retorna tokens da sequência de entrada BERT
e seus IDs de segmento correspondentes.


In [14]:
#@save
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """Get tokens of the BERT input sequence and their segment IDs."""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0 and 1 are marking segment A and B, respectively
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

O BERT escolhe o codificador Transformer como sua arquitetura bidirecional.
Comum no codificador do transformador,
embeddings posicionais são adicionados em cada posição da sequência de entrada BERT.
Entretanto, diferente do codificador Transformer original,
O BERT usa embeddings posicionais *aprendíveis*.
Para resumir, :numref:`fig_bert-input` mostra que
as incorporações da sequência de entrada BERT são a soma
dos embeddings de token, embeddings de segmento e embeddings posicionais.

![Os embeddings da sequência de entrada BERT são a soma
dos embeddings de token, embeddings de segmento e embeddings posicionais.](https://github.com/d2l-ai/d2l-pytorch-colab/blob/master/img/bert-input.svg?raw=1)
:label:`fig_bert-entrada`

A seguinte [**`classe BERTEncoder`**] é semelhante à classe `TransformerEncoder`
conforme implementado em :numref:`sec_transformer`.
Diferente de `TransformerEncoder`, `BERTEncoder` usa
embeddings de segmento e embeddings posicionais aprendíveis.


In [15]:
#@save
class BERTEncoder(nn.Module):
    """BERT encoder."""
    def __init__(self, vocab_size, num_hiddens, ffn_num_hiddens, num_heads,
                 num_blks, dropout, max_len=1000, **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)
        self.segment_embedding = nn.Embedding(2, num_hiddens)
        self.blks = nn.Sequential()
        for i in range(num_blks):
            self.blks.add_module(f"{i}", d2l.TransformerEncoderBlock(
                num_hiddens, ffn_num_hiddens, num_heads, dropout, True))
        # In BERT, positional embeddings are learnable, thus we create a
        # parameter of positional embeddings that are long enough
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len,
                                                      num_hiddens))

    def forward(self, tokens, segments, valid_lens):
        # Shape of `X` remains unchanged in the following code snippet:
        # (batch size, max sequence length, `num_hiddens`)
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        X = X + self.pos_embedding[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X

Suponha que o tamanho do vocabulário seja 10000.
Para demonstrar a [**inferência de `BERTEncoder`**] para a frente,
vamos criar uma instância dele e inicializar seus parâmetros.


In [16]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
ffn_num_input, num_blks, dropout = 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, ffn_num_hiddens, num_heads,
                      num_blks, dropout)

Definimos `tokens` como 2 sequências de entrada BERT de comprimento 8,
onde cada token é um índice do vocabulário.
A inferência direta de `BERTEncoder` com a entrada `tokens`
retorna o resultado codificado onde cada token é representado por um vetor
cujo comprimento é predefinido pelo hiperparâmetro `num_hiddens`.
Este hiperparâmetro é geralmente chamado de *tamanho oculto*
(número de unidades ocultas) do codificador do transformador.


In [17]:
tokens = torch.randint(0, vocab_size, (2, 8))
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
encoded_X = encoder(tokens, segments, None)
encoded_X.shape

torch.Size([2, 8, 768])

## Tarefas de pré-treinamento
:label:`tarefas_de_pré-treinamento_subsec_bert`

A inferência direta de `BERTEncoder` fornece a representação BERT
de cada token do texto de entrada e do inserido
tokens especiais “&lt;cls&gt;” e “&lt;seq&gt;”.
A seguir, usaremos essas representações para calcular a função de perda
para pré-treinamento BERT.
O pré-treinamento é composto pelas duas tarefas seguintes:
modelagem de linguagem mascarada e previsão da próxima frase.

### [**Modelagem de Linguagem Mascarada**]
:rótulo:`subsec_mlm`

Conforme ilustrado em :numref:`sec_language-model`,
um modelo de linguagem prevê um token usando o contexto à sua esquerda.
Para codificar o contexto bidirecionalmente para representar cada token,
BERT mascara tokens aleatoriamente e usa tokens do contexto bidirecional para
prever os tokens mascarados de forma autossupervisionada.
Essa tarefa é chamada de *modelo de linguagem mascarada*.

Nesta tarefa de pré-treinamento,
15% dos tokens serão selecionados aleatoriamente como tokens mascarados para previsão.
Para prever um token mascarado sem trapacear usando o rótulo,
uma abordagem direta é sempre substituí-lo por um token especial “&lt;mask&gt;” na sequência de entrada BERT.
No entanto, o token especial artificial “&lt;mask&gt;” nunca aparecerá
em ajuste fino.
Para evitar tal incompatibilidade entre o pré-treinamento e o ajuste fino,
se um token for mascarado para previsão (por exemplo, "ótimo" é selecionado para ser mascarado e previsto em "este filme é ótimo"),
na entrada será substituído por:

* um token especial “&lt;mask&gt;” para 80% do tempo (por exemplo, "este filme é ótimo" se torna "este filme é &lt;mask&gt;");
* um token aleatório para 10% do tempo (por exemplo, "este filme é ótimo" se torna "este filme é uma bebida");
* o token de rótulo inalterado por 10% do tempo (por exemplo, "este filme é ótimo" se torna "este filme é ótimo").

Observe que durante 10% de 15% do tempo um token aleatório é inserido.
Esse ruído ocasional incentiva o BERT a ser menos tendencioso em relação ao token mascarado (especialmente quando o token de rótulo permanece inalterado) em sua codificação de contexto bidirecional.

Implementamos a seguinte classe `MaskLM` para prever tokens mascarados
na tarefa do modelo de linguagem mascarada do pré-treinamento de BERT.
A previsão usa um MLP de camada oculta (`self.mlp`).
Na inferência direta, são necessárias duas entradas:
o resultado codificado de `BERTEncoder` e as posições do token para previsão.
A saída são os resultados da previsão nessas posições.


In [18]:
#@save
class MaskLM(nn.Module):
    """The masked language model task of BERT."""
    def __init__(self, vocab_size, num_hiddens, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        self.mlp = nn.Sequential(nn.LazyLinear(num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.LazyLinear(vocab_size))

    def forward(self, X, pred_positions):
        num_pred_positions = pred_positions.shape[1]
        pred_positions = pred_positions.reshape(-1)
        batch_size = X.shape[0]
        batch_idx = torch.arange(0, batch_size)
        # Suppose that `batch_size` = 2, `num_pred_positions` = 3, then
        # `batch_idx` is `torch.tensor([0, 0, 0, 1, 1, 1])`
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)
        masked_X = X[batch_idx, pred_positions]
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat

Para demonstrar [**a inferência direta de `MaskLM`**],
criamos sua instância `mlm` e a inicializamos.
Lembre-se de `encoded_X` da inferência direta de `BERTEncoder`
representa 2 sequências de entrada BERT.
Definimos `mlm_positions` como os 3 índices a serem previstos em qualquer sequência de entrada BERT de `encoded_X`.
A inferência direta de `mlm` retorna resultados de previsão `mlm_Y_hat`
em todas as posições mascaradas `mlm_positions` de `encoded_X`.
Para cada previsão, o tamanho do resultado é igual ao tamanho do vocabulário.


In [19]:
mlm = MaskLM(vocab_size, num_hiddens)
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
mlm_Y_hat = mlm(encoded_X, mlm_positions)
mlm_Y_hat.shape

torch.Size([2, 3, 10000])

Com os rótulos de verdade básica `mlm_Y` dos tokens previstos `mlm_Y_hat` sob máscaras,
podemos calcular a perda de entropia cruzada da tarefa do modelo de linguagem mascarada no pré-treinamento de BERT.


In [20]:
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
loss = nn.CrossEntropyLoss(reduction='none')
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l.shape

torch.Size([6])

### [**Previsão da próxima frase**]
:rótulo:`subsec_nsp`

Embora a modelagem de linguagem mascarada seja capaz de codificar contexto bidirecional
para representar palavras, não modela explicitamente a relação lógica
entre pares de texto.
Para ajudar a entender a relação entre duas sequências de texto,
O BERT considera uma tarefa de classificação binária, *previsão da próxima frase*, em seu pré-treinamento.
Ao gerar pares de frases para pré-treinamento,
na metade do tempo são de fato frases consecutivas com o rótulo "Verdadeiro";
enquanto na outra metade do tempo a segunda frase é amostrada aleatoriamente do corpus com o rótulo "Falso".

A seguinte classe `NextSentencePred` usa um MLP de camada oculta
para prever se a segunda frase é a próxima frase da primeira
na sequência de entrada do BERT.
Devido à autoatenção no codificador do transformador,
a representação BERT do token especial “&lt;cls&gt;”
codifica as duas frases da entrada.
Portanto, a camada de saída (`self.output`) do classificador MLP recebe `X` como entrada,
onde `X` é a saída da camada oculta MLP cuja entrada é o token codificado “&lt;cls&gt;”.


In [21]:
#@save
class NextSentencePred(nn.Module):
    """The next sentence prediction task of BERT."""
    def __init__(self, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        self.output = nn.LazyLinear(2)

    def forward(self, X):
        # `X` shape: (batch size, `num_hiddens`)
        return self.output(X)

Podemos ver que [**a inferência direta de uma instância `NextSentencePred`**]
retorna previsões binárias para cada sequência de entrada BERT.


In [22]:
# PyTorch by default will not flatten the tensor as seen in mxnet where, if
# flatten=True, all but the first axis of input data are collapsed together
encoded_X = torch.flatten(encoded_X, start_dim=1)
# input_shape for NSP: (batch size, `num_hiddens`)
nsp = NextSentencePred()
nsp_Y_hat = nsp(encoded_X)
nsp_Y_hat.shape

torch.Size([2, 2])

A perda de entropia cruzada das duas classificações binárias também pode ser calculada.


In [23]:
nsp_y = torch.tensor([0, 1])
nsp_l = loss(nsp_Y_hat, nsp_y)
nsp_l.shape

torch.Size([2])

É digno de nota que todos os rótulos em ambas as tarefas de pré-treinamento acima mencionadas
pode ser obtido trivialmente do corpus de pré-treinamento sem esforço de rotulagem manual.
O BERT original foi pré-treinado na concatenação do BookCorpus :cite:`Zhu.Kiros.Zemel.ea.2015`
e Wikipédia em inglês.
Esses dois corpora de texto são enormes:
eles têm 800 milhões de palavras e 2,5 bilhões de palavras, respectivamente.


## [**Juntando tudo**]

Ao pré-treinar o BERT, a função de perda final é uma combinação linear de
ambas as funções de perda para modelagem de linguagem mascarada e previsão da próxima frase.
Agora podemos definir a classe `BERTModel` instanciando as três classes
`BERTEncoder`, `MaskLM` e `NextSentencePred`.
A inferência direta retorna as representações BERT codificadas `encoded_X`,
previsões de modelagem de linguagem mascarada `mlm_Y_hat`,
e as próximas previsões de frases `nsp_Y_hat`.


In [24]:
#@save
class BERTModel(nn.Module):
    """The BERT model."""
    def __init__(self, vocab_size, num_hiddens, ffn_num_hiddens,
                 num_heads, num_blks, dropout, max_len=1000):
        super(BERTModel, self).__init__()
        self.encoder = BERTEncoder(vocab_size, num_hiddens, ffn_num_hiddens,
                                   num_heads, num_blks, dropout,
                                   max_len=max_len)
        self.hidden = nn.Sequential(nn.LazyLinear(num_hiddens),
                                    nn.Tanh())
        self.mlm = MaskLM(vocab_size, num_hiddens)
        self.nsp = NextSentencePred()

    def forward(self, tokens, segments, valid_lens=None, pred_positions=None):
        encoded_X = self.encoder(tokens, segments, valid_lens)
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # The hidden layer of the MLP classifier for next sentence prediction.
        # 0 is the index of the '<cls>' token
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :]))
        return encoded_X, mlm_Y_hat, nsp_Y_hat

## Resumo

* Modelos de incorporação de palavras como word2vec e GloVe são independentes de contexto. Eles atribuem o mesmo vetor pré-treinado à mesma palavra, independentemente do contexto da palavra (se houver). É difícil para eles lidar bem com polissemia ou semântica complexa em línguas naturais.
* Para representações de palavras sensíveis ao contexto, como ELMo e GPT, as representações de palavras dependem de seus contextos.
* O ELMo codifica o contexto bidirecionalmente, mas usa arquiteturas específicas para cada tarefa (no entanto, é praticamente não trivial criar uma arquitetura específica para cada tarefa de processamento de linguagem natural); enquanto o GPT é independente de tarefa, mas codifica o contexto da esquerda para a direita.
* O BERT combina o melhor dos dois mundos: ele codifica o contexto bidirecionalmente e requer mudanças mínimas de arquitetura para uma ampla gama de tarefas de processamento de linguagem natural.
* Os embeddings da sequência de entrada BERT são a soma dos embeddings de tokens, embeddings de segmentos e embeddings posicionais.
* O pré-treinamento BERT é composto de duas tarefas: modelagem de linguagem mascarada e previsão da próxima frase. A primeira é capaz de codificar contexto bidirecional para representar palavras, enquanto a última modela explicitamente o relacionamento lógico entre pares de texto.


## Exercícios

1. Todas as outras coisas sendo iguais, um modelo de linguagem mascarada exigirá mais ou menos etapas de pré-treinamento para convergir do que um modelo de linguagem da esquerda para a direita? Por quê?
1. Na implementação original do BERT, a rede de feed-forward posicional em `BERTEncoder` (via `d2l.TransformerEncoderBlock`) e a camada totalmente conectada em `MaskLM` usam a unidade linear de erro gaussiano (GELU) :cite:`Hendrycks.Gimpel.2016` como a função de ativação. Pesquise sobre a diferença entre GELU e ReLU.


[Discussões](https://discuss.d2l.ai/t/1490)
